In [2]:
import cv2 
import numpy as np 
import smtplib
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText 
import os
from email import encoders
from email.mime.base import MIMEBase

In [3]:
def send_email(image_paths):
    # Set up the email parameters
    sender_email = "-"
    receiver_email = "-"
    password = "your-passkey"
    subject = "Intruder Alert!"
    message = "An intruder has been detected in the area!"

    # Set up the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))

    # Attach images to the email
    for image_path in image_paths:
        attachment = open(image_path, 'rb')
        img = MIMEBase('application', 'octet-stream')
        img.set_payload((attachment).read())
        encoders.encode_base64(img)
        img.add_header('Content-Disposition', "attachment; filename= %s" % os.path.basename(image_path))
        msg.attach(img)

    # Send the email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)
    text = msg.as_string()
    server.sendmail(sender_email, receiver_email, text)
    server.quit()

In [4]:
def check_intruder():
    global contours, rect
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 900:
            continue
        if x < rect[0] or y < rect[1] or x > rect[0] + rect[2] or y > rect[1] + rect[3]:
            return True
    return False



In [5]:
def click_event(event, x, y, flags, param):
    global draw,a,b  
    if event==cv2.EVENT_LBUTTONDOWN:
        a,b = x,y
        draw =1
    elif event == cv2.EVENT_LBUTTONDOWN:
        if draw == 1:
            frame = frame1
            # cv2. rectangle(frmae1, (a,b), (x,y), (0,255,0), 1)
            cv2.imshow("frame", frame1)
            cv2.waitKey(0)
            # frame1=frame

    elif event == cv2.EVENT_LBUTTONUP:
        cv2.rectangle(frame1, (a,b), (x,y), (0,0,255), 1)
        global rect 
        rect = a,b,x,y
        draw = 0
        cv2.imshow("frame", frame1)
        cv2.waitKey(0)
        

In [6]:
global draw, frame1, rect
draw = 0 # 1 while dragging the mouse pointer
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Couldn't open the webcam.")
    exit()

ret, frame1 = cap.read()
if not ret:
    print("Error: Couldn't read the first frame.")
    exit()

ret,frame1=cap.read()
ret,frame2=cap.read()
rect = 0,0,frame1.shape[1],frame1.shape[0]
cv2.imshow('frame',frame1)
cv2.setMouseCallback('frame', click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
last_email_time = 0
delay_between_emails = 60  # Set the delay in seconds (e.g., 60 seconds)

email_sent= False
image_path_list = []

while cap.isOpened():
    try:

        diff = cv2.absdiff(frame1, frame2)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        thresh = cv2.resize(thresh, (frame1.shape[1], frame1.shape[0]))
        dilated = cv2.dilate(thresh, None, iterations=3)

        crop = dilated[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
        cv2.imshow("mask", dilated)

        contours, _ = cv2.findContours(crop, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        cv2.rectangle(frame1, (rect[0], rect[1]), (rect[2], rect[3]), (0, 0, 255), 2)

        for contour in contours:
                    (x, y, w, h) = cv2.boundingRect(contour)
                    if cv2.contourArea(contour) < 900:
                        continue
                    cv2.rectangle(frame1, (rect[0] + x, rect[1] + y), (rect[0] + x + w, rect[1] + y + h), (0, 255, 0), 2)
                    cv2.putText(frame1, "Intruder Found", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        intruder_detected = check_intruder()

        if intruder_detected and not email_sent:
            # Check if enough time has passed since the last email
            current_time = time.time()
            if current_time - last_email_time >= delay_between_emails:
                # Capture the intruder image
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_path = f"intruder_{timestamp}.png"
                cv2.imwrite(image_path, frame1)
                image_path_list.append(image_path)

                # Send an email notification to the owner with attached images
                send_email(image_path_list)

                # Update the last email time
                last_email_time = current_time
                email_sent = True  # Set the flag to indicate that an email has been sent

        elif not intruder_detected:
            # Reset the email_sent flag if no intruder is detected
            email_sent = False
            image_path_list = []

        cv2.imshow('feed', frame1)
        frame1 = frame2
        ret, frame2 = cap.read()

        if cv2.waitKey(40) == 27:
            break

    except Exception as e:
        print("Error:", e)
        break

cv2.destroyAllWindows()
cap.release()